In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
import tensorflow as tf

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from google.colab import files

In [0]:
sess = None
graph=None
input_tensor = None
n_neighbours = None
predicted_labels = None
predicted_similarities = None
output_node_names= ["predicted_classes","predicted_nearest_idx"]

In [0]:
from keras.models import load_model

In [0]:
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

In [0]:
feature_extractor = load_model("/content/gdrive/My Drive/pdd_model_new/feature_extractor_all_150epochs.h5")

W0723 07:48:28.309381 140054012438400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 07:48:28.331648 140054012438400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0723 07:48:28.368620 140054012438400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0723 07:48:28.369786 140054012438400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0723 07:48:28.370889 140054012

In [0]:
#N_CLASSES = np.unique(train_dataset['target']).size
N_CLASSES=30


In [0]:
base_model=feature_extractor
x=base_model.output
preds=Dense(N_CLASSES, activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=preds)

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 247, 247, 32)      9632      
_________________________________________________________________
activation_1 (Activation)    (None, 247, 247, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 247, 247, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 123, 123, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 117, 117, 64)      100416    
_________________________________________________________________
activation_2 (Activation)    (None, 117, 117, 64)      0         
__________

In [0]:
for layer in model.layers[:-1]:
    layer.trainable=False

In [0]:
!rm -r -f pdd_new
!git clone https://github.com/Kaliostrogoblin/PDD.git

Cloning into 'pdd_new'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 268 (delta 161), reused 224 (delta 134), pack-reused 0
Receiving objects: 100% (268/268), 53.90 MiB | 47.83 MiB/s, done.
Resolving deltas: 100% (161/161), done.


In [0]:
import os
os.chdir('PDD')
# verify if we are in correct directory
os.listdir()

['pdd', '.gitignore', 'examples', 'server', '.git', 'README.md']

In [0]:
from pdd.datasets.allcrops import load_data

train_data, test_data = load_data(split_on_train_test=True, random_state=13)

100%|███████████████████| 15/15 [00:00<00:00, 154.64it/s]

Splitting on train and test...


In [0]:
from pdd.utils.data_utils import create_dataset_from_dir

train_dataset = create_dataset_from_dir(train_data, shuffle=True)
test_dataset = create_dataset_from_dir(test_data, shuffle=True)

In [0]:
train_dataset.keys()

dict_keys(['data', 'target', 'target_names'])

In [0]:
test_dataset.keys()

dict_keys(['data', 'target', 'target_names'])

In [0]:
print (test_dataset['target_names'])

['Grape_Black rot', 'Grape_Esca', 'Grape_Powdery mildew', 'Wheat_Black chaff', 'Corn_Eyespot', 'Wheat_Powdery mildew', 'Grape_Chlorosis', 'Corn_Southern rust', 'Corn_Northern leaf blight', 'Corn_Downy mildew', 'Wheat_Yellow rust', 'Grape_Healthy', 'Wheat_Brown rust', 'Corn_Healthy', 'Wheat_Healthy']


In [0]:
print (train_dataset['target_names'])

['Grape_Black rot', 'Grape_Esca', 'Grape_Powdery mildew', 'Wheat_Black chaff', 'Corn_Eyespot', 'Wheat_Powdery mildew', 'Grape_Chlorosis', 'Corn_Southern rust', 'Corn_Northern leaf blight', 'Corn_Downy mildew', 'Wheat_Yellow rust', 'Grape_Healthy', 'Wheat_Brown rust', 'Corn_Healthy', 'Wheat_Healthy']


In [0]:
train_dataset['data'], train_dataset['target']

(array([[[[0.88627451, 0.93333333, 0.84705882],
          [0.85490196, 0.90980392, 0.76862745],
          [0.78431373, 0.85490196, 0.61960784],
          ...,
          [0.22745098, 0.52156863, 0.05882353],
          [0.28627451, 0.5372549 , 0.16078431],
          [0.68627451, 0.81176471, 0.62745098]],
 
         [[0.76078431, 0.83921569, 0.49019608],
          [0.67843137, 0.76078431, 0.39607843],
          [0.6       , 0.69411765, 0.30980392],
          ...,
          [0.35294118, 0.58823529, 0.23529412],
          [0.70196078, 0.81568627, 0.60392157],
          [0.94509804, 1.        , 0.89803922]],
 
         [[0.57647059, 0.69803922, 0.16078431],
          [0.54901961, 0.66666667, 0.15686275],
          [0.54901961, 0.66666667, 0.18823529],
          ...,
          [0.73333333, 0.85882353, 0.66666667],
          [0.99607843, 1.        , 0.95294118],
          [0.99607843, 1.        , 0.96862745]],
 
         ...,
 
         [[0.13333333, 0.24313725, 0.01568627],
          [0.13333

In [0]:
 (test_dataset['data'], test_dataset['target'])

(array([[[[0.04705882, 0.04705882, 0.03921569],
          [0.03529412, 0.03529412, 0.02745098],
          [0.03137255, 0.03137255, 0.02352941],
          ...,
          [0.42745098, 0.58823529, 0.11764706],
          [0.39607843, 0.56470588, 0.10980392],
          [0.32941176, 0.49411765, 0.05490196]],
 
         [[0.03137255, 0.03137255, 0.02352941],
          [0.01568627, 0.01568627, 0.00784314],
          [0.01568627, 0.01568627, 0.00784314],
          ...,
          [0.45490196, 0.61176471, 0.12156863],
          [0.37254902, 0.52941176, 0.08235294],
          [0.36470588, 0.51764706, 0.10196078]],
 
         [[0.02745098, 0.02745098, 0.01960784],
          [0.01568627, 0.01568627, 0.00784314],
          [0.00784314, 0.00784314, 0.        ],
          ...,
          [0.47843137, 0.62745098, 0.14901961],
          [0.41960784, 0.57254902, 0.11372549],
          [0.5372549 , 0.68627451, 0.25098039]],
 
         ...,
 
         [[0.12941176, 0.1372549 , 0.13333333],
          [0.12156

In [0]:
from keras.optimizers import SGD
!pip uninstall keras
!pip install git+git://github.com/keras-team/keras.git

Uninstalling Keras-2.2.4:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.2.4.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.2.4
  Cloning git://github.com/keras-team/keras.git to /tmp/pip-req-build-rn7h_lim
  Running command git clone -q git://github.com/keras-team/keras.git /tmp/pip-req-build-rn7h_lim
  Stored in directory: /tmp/pip-ephem-wheel-cache-iln7w7z3/wheels/e6/02/ad/5e8e1a5824af71082e2260fe7e2eaa1b745c34706e6ff0a14b
Successfully built Keras


In [0]:
sgd = SGD(lr=0.01, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.summary()

W0723 07:50:24.196166 140054012438400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 247, 247, 32)      9632      
_________________________________________________________________
activation_1 (Activation)    (None, 247, 247, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 247, 247, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 123, 123, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 117, 117, 64)      100416    
_________________________________________________________________
activation_2 (Activation)    (None, 117, 117, 64)      0         
__________

In [0]:
from keras.utils import to_categorical
train_categorical = np.eye(N_CLASSES)[train_dataset['target']]
test_categorical  = np.eye(N_CLASSES)[test_dataset['target']]

In [0]:
history=model.fit(x=train_dataset['data'],
          y=train_categorical, 
          batch_size=32,
          epochs=100,
          validation_data=(test_dataset['data'], test_categorical),
          shuffle=True)

W0723 07:50:30.233540 140054012438400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 447 samples, validate on 70 samples
Epoch 1/100
447/447 [==============================] - 7s 15ms/step - loss: 2.1599 - acc: 0.3714 - val_loss: 0.6813 - val_acc: 0.8571
Epoch 2/100
447/447 [==============================] - 1s 3ms/step - loss: 0.4368 - acc: 0.9128 - val_loss: 0.2650 - val_acc: 0.9000
Epoch 3/100
447/447 [==============================] - 1s 3ms/step - loss: 0.2781 - acc: 0.9374 - val_loss: 0.2161 - val_acc: 0.9286
Epoch 4/100
447/447 [==============================] - 1s 3ms/step - loss: 0.2059 - acc: 0.9553 - val_loss: 0.1492 - val_acc: 0.9714
Epoch 5/100
447/447 [==============================] - 1s 3ms/step - loss: 0.1748 - acc: 0.9508 - val_loss: 0.1433 - val_acc: 0.9714
Epoch 6/100
447/447 [==============================] - 1s 3ms/step - loss: 0.1660 - acc: 0.9485 - val_loss: 0.1696 - val_acc: 0.9429
Epoch 7/100
447/447 [==============================] - 1s 3ms/step - loss: 0.1436 - acc: 0.9664 - val_loss: 0.1354 - val_acc: 0.9857
Epoch 8/100
447/447 [==

In [0]:
print("Saving feature extractor...")
model.save('/content/gdrive/My Drive/pdd_classifier_all_150epochs_100epochs.h5')

Saving feature extractor...
